In [1]:
#imports

import pandas as pd
import numpy as np
import os

import datetime

from prepare import zillow_pipeline

In [2]:
df = zillow_pipeline()

In [3]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state
0,4,3.5,3100,1023282,1998,Orange,CA
1,2,1.0,1465,464000,1967,Ventura,CA
2,3,2.0,1243,564778,1962,Orange,CA
3,4,3.0,2376,145143,1970,Los Angeles,CA
4,4,3.0,2962,773303,1950,Los Angeles,CA


In [4]:
def categorize_col(df, target_column):
    # Find percentiles for the target column
    percentiles = df[target_column].quantile([0.01, 0.05, 0.10, 0.20, 0.25, 0.75, 0.80, 0.90, 0.95, 0.99])
    
    # Define the bins and labels
    bins = [0] + list(percentiles) + [df[target_column].max()]
    labels = ['bottom 1%', 'bottom 5%', 'bottom 10%', 'bottom 20%', 'bottom 25%',
              'middle 50%', 'top 25%', 'top 20%', 'top 10%', 'top 5%', 'top 1%']

    # Create a new column based on the percentiles
    df[f'group_{target_column}'] = pd.cut(df[target_column], bins=bins, labels=labels)
    
    return df

def group_counts(target_column):
    # Count the occurrences of each category in the 'size_group' column
    group_counts_l = df[f'group_{target_column}'].value_counts().sort_index()

    # Print the category names and their corresponding counts
    for category, count in group_counts_l.items():
        print(f'{category} of {target_column}: {count}')


def group_ranges(target_column):
    # Group the DataFrame by 'size_group' and calculate the minimum and maximum values for 'area'
    group_ranges = df.groupby(f'group_{target_column}')[target_column].agg(['min', 'max'])

    # Print the size_group ranges
    for group_, (min_, max_) in group_ranges.iterrows():
        print(f'{group_} of {target_column}: {min_:,}-{max_:,}')

In [5]:
# Assuming df is your original DataFrame
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

current_year = datetime.datetime.now().year
df['age'] = current_year - df['year']

#df['price_per_sqft'] = df['tax_value'] / df['area']

#df['tax_rate'] = df['taxamount'] / df['tax_value']

#df = pd.get_dummies(df, columns=['fips'], prefix='county')

df['bdrm_area_ratio'] = df['area'] / df['bedrooms']

df['bath_bdrm_ratio'] = df['bathrooms'] / df['bedrooms']

In [6]:
print(f'Min:{df.bedrooms.min()}, Max:{df.bedrooms.max()}')

Min:1, Max:14


In [7]:
print(f'Min:{df.bathrooms.min()}, Max:{df.bathrooms.max()}')

Min:1.0, Max:18.0


In [8]:
print(f'Min:{df.year.min()}, Max:{df.year.max()}')

Min:1878, Max:2016


In [9]:
print(f'Min:{df.age.min()}, Max:{df.age.max()}')

Min:7, Max:145


In [10]:
df = categorize_col(df, 'value')
group_ranges('value')

bottom 1% of value: 1,000-37,435
bottom 5% of value: 37,437-61,666
bottom 10% of value: 61,667-84,238
bottom 20% of value: 84,242-155,114
bottom 25% of value: 155,139-193,998
middle 50% of value: 194,000-618,794
top 25% of value: 618,900-701,300
top 20% of value: 701,390-1,010,867
top 10% of value: 1,011,276-1,471,865
top 5% of value: 1,472,047-3,092,692
top 1% of value: 3,092,695-49,061,236


In [12]:
group_counts('value')

bottom 1% of value: 524
bottom 5% of value: 2089
bottom 10% of value: 2612
bottom 20% of value: 5225
bottom 25% of value: 2613
middle 50% of value: 26121
top 25% of value: 2613
top 20% of value: 5224
top 10% of value: 2612
top 5% of value: 2090
top 1% of value: 523


In [14]:
df = categorize_col(df, 'area')
group_ranges('area')

bottom 1% of area: 152-699
bottom 5% of area: 700-896
bottom 10% of area: 897-1,032
bottom 20% of area: 1,033-1,200
bottom 25% of area: 1,201-1,268
middle 50% of area: 1,269-2,305
top 25% of area: 2,306-2,510
top 20% of area: 2,511-3,122
top 10% of area: 3,123-3,730
top 5% of area: 3,731-5,497
top 1% of area: 5,500-21,929


In [20]:
df = categorize_col(df, 'age')
group_ranges('age')

bottom 1% of age: 7-12
bottom 5% of age: 13-20
bottom 10% of age: 21-25
bottom 20% of age: 26-37
bottom 25% of age: 38-44
middle 50% of age: 45-73
top 25% of age: 74-75
top 20% of age: 76-90
top 10% of age: 91-99
top 5% of age: 100-113
top 1% of age: 114-145


In [21]:
df = categorize_col(df, 'bdrm_area_ratio')
group_ranges('bdrm_area_ratio')

bottom 1% of bdrm_area_ratio: 30.4-296.0
bottom 5% of bdrm_area_ratio: 296.3333333333333-340.0
bottom 10% of bdrm_area_ratio: 340.25-368.5
bottom 20% of bdrm_area_ratio: 368.6-411.0
bottom 25% of bdrm_area_ratio: 411.1666666666667-430.0
middle 50% of bdrm_area_ratio: 430.2-668.0
top 25% of bdrm_area_ratio: 668.2-709.5
top 20% of bdrm_area_ratio: 709.6666666666666-845.5
top 10% of bdrm_area_ratio: 845.6666666666666-981.8
top 5% of bdrm_area_ratio: 982.0-1,325.0
top 1% of bdrm_area_ratio: 1,325.3333333333333-3,236.0


In [45]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,bdrm_area_ratio,bath_bdrm_ratio,group_value
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,top 10%
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,middle 50%
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,middle 50%
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,bottom 20%
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,top 20%


In [46]:
df.shape

(52246, 12)

In [35]:
# Drop rows with 'top 1%' or 'bottom 1%' in 'group_value'
df = df[(df['group_value'] != 'top 1%') & (df['group_value'] != 'bottom 1%')]

In [50]:
df.shape

(51199, 12)

In [37]:
52246 - 51199

1047

In [21]:
# Define the age group categories
age_bins = [-1, 4, 9, 19, 39, 59, 79, 200]
age_labels = ['Newest', 'Very_New', 'New', 'Mid', 'Old', 'Very_Old', 'Oldest']

# Create the "age_group" column
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old


In [34]:
# Count the occurrences of each category in the 'size_group' column
age_group_counts = df['age_group'].value_counts().sort_index()

# Print the category names and their corresponding counts
for category, count in age_group_counts.items():
    print(f'{category}: {count}')

Newest: 0
Very_New: 248
New: 2347
Mid: 9006
Old: 11713
Very_Old: 20622
Oldest: 8310


In [24]:
# Find percentiles for square footage
percentiles = df['area'].quantile([0.01, 0.05, 0.20, 0.80, 0.95, 0.99])

# Define the categories
df['size_group'] = pd.cut(
    df['area'],
    bins=[0, percentiles[0.01], percentiles[0.05], percentiles[0.20], percentiles[0.80], percentiles[0.95], percentiles[0.99], df['area'].max()],
    labels=['Tiny', 'Small', 'Medium', 'Average', 'Large', 'Very Large', 'Largest']
)

In [25]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group,size_group
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid,Large
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old,Average
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old,Average
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old,Average
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old,Large


In [26]:
# Count the occurrences of each category in the 'size_group' column
size_group_counts = df['size_group'].value_counts().sort_index()

# Print the category names and their corresponding counts
for category, count in size_group_counts.items():
    print(f'{category}: {count}')

Tiny: 523
Small: 2114
Medium: 7829
Average: 31351
Large: 7819
Very Large: 2088
Largest: 522


In [27]:

# Group the DataFrame by 'size_group' and calculate the minimum and maximum values for 'area'
size_group_ranges = df.groupby('size_group')['area'].agg(['min', 'max'])

# Print the size_group ranges
for size_group, (min_area, max_area) in size_group_ranges.iterrows():
    print(f'{size_group}: {min_area}-{max_area} sqft')

Tiny: 152-699 sqft
Small: 700-896 sqft
Medium: 897-1200 sqft
Average: 1201-2510 sqft
Large: 2511-3730 sqft
Very Large: 3731-5497 sqft
Largest: 5500-21929 sqft


In [29]:
# Calculate the percentiles
percentiles = df['area'].quantile([0.01, 0.05, 0.32, 0.68, 0.95, 0.99])

# Define the categories
df['sqft_category'] = pd.cut(
    df['area'],
    bins=[0, percentiles[0.01], percentiles[0.05], percentiles[0.32], percentiles[0.68], percentiles[0.95], percentiles[0.99], df['area'].max()],
    labels=['smallest', 'very_small', 'small', 'Average', 'large', 'very_large', 'Largest']
)


In [30]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group,size_group,sqft_category
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid,Large,Large
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old,Average,Average
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old,Average,Medium
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old,Average,Large
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old,Large,Large


In [31]:
# Count the occurrences of each category in the 'size_group' column
size_group_counts = df['sqft_category'].value_counts().sort_index()

# Print the category names and their corresponding counts
for category, count in size_group_counts.items():
    print(f'{category}: {count}')

Tiny: 523
Small: 2114
Medium: 14097
Average: 18797
Large: 14105
very_large: 2088
Largest: 522


In [32]:

# Group the DataFrame by 'size_group' and calculate the minimum and maximum values for 'area'
size_group_ranges = df.groupby('sqft_category')['area'].agg(['min', 'max'])

# Print the size_group ranges
for size_group, (min_area, max_area) in size_group_ranges.iterrows():
    print(f'{size_group}: {min_area}-{max_area} sqft')

Tiny: 152-699 sqft
Small: 700-896 sqft
Medium: 897-1374 sqft
Average: 1375-2072 sqft
Large: 2073-3730 sqft
very_large: 3731-5497 sqft
Largest: 5500-21929 sqft
